<span style="font-weight: 600; background-color: rgb(41, 42, 45); color: rgb(248, 250, 255); font-family: quote-cjk-patch, Inter, system-ui, -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Roboto, &quot;Noto Sans&quot;, Ubuntu, Cantarell, &quot;Helvetica Neue&quot;, Oxygen, &quot;Open Sans&quot;, sans-serif; font-size: 27.432px;">Premier League Data Warehouse - Data Validation &amp; Quality Checks</span>

## 📌 Introduction

This SQL Notebook contains data validation and quality checks for the Premier League Data Warehouse to ensure:  
✅ Referential Integrity (Foreign Key constraints)  
✅ Data Completeness (No NULLs in critical fields)  
✅ Data Consistency (Valid ranges, no duplicates)  
✅ Business Logic (e.g., `goals` ≥ 0, `possession` between 0-100)

## 🔍 1. Referential Integrity Checks

### 1.1 Players with No Team Reference (Orphaned Players)

In [1]:
SELECT 
    p.player_name,
    p.team_name
FROM 
    Players p
LEFT JOIN 
    Teams t ON p.team_name = t.team_name
WHERE 
    t.team_name IS NULL;

(0 rows affected)

Total execution time: 00:00:00.302

player_name,team_name


### 1.2 Fixtures with Invalid Team References

In [2]:
SELECT 
    f.week,
    f.home_team,
    f.away_team
FROM 
    Fixtures f
LEFT JOIN 
    Teams ht ON f.home_team = ht.team_name
LEFT JOIN 
    Teams at ON f.away_team = at.team_name
WHERE 
    ht.team_name IS NULL OR at.team_name IS NULL;

(0 rows affected)

Total execution time: 00:00:00.095

week,home_team,away_team


## 📊 2. Data Completeness Checks

### 2.1 Missing Player Stats

In [3]:
SELECT 
    p.player_name
FROM 
    Players p
LEFT JOIN 
    Player_Stats ps ON p.player_name = ps.player_name
WHERE 
    ps.player_name IS NULL;

(0 rows affected)

Total execution time: 00:00:00.125

player_name


### 2.2 NULL Values in Critical Columns

In [4]:
-- Check for NULLs in Player_Stats
SELECT 
    player_name,
    played,
    goals,
    assists
FROM 
    Player_Stats
WHERE 
    played IS NULL 
    OR goals IS NULL 
    OR assists IS NULL;

(0 rows affected)

Total execution time: 00:00:00.148

player_name,played,goals,assists


## 🔢 3. Data Consistency & Business Logic

### 3.1 Negative Stats (Invalid Values)

In [5]:
SELECT 
    player_name,
    goals,
    assists,
    yellow,
    red
FROM 
    Player_Stats
WHERE 
    goals < 0 
    OR assists < 0 
    OR yellow < 0 
    OR red < 0;

(0 rows affected)

Total execution time: 00:00:00.018

player_name,goals,assists,yellow,red


### 3.2 Possession Percentage Out of Range (0-100%)

In [6]:
SELECT 
    team_name,
    possession
FROM 
    Team_Stats
WHERE 
    possession < 0 OR possession > 100;

(0 rows affected)

Total execution time: 00:00:00.088

team_name,possession


### 3.3 Duplicate Players

In [7]:
SELECT 
    player_name,
    COUNT(*) as duplicate_count
FROM 
    Players
GROUP BY 
    player_name
HAVING 
    COUNT(*) > 1;

(0 rows affected)

Total execution time: 00:00:00.089

player_name,duplicate_count


## 📈 4. Cross-Table Validation

### 4.1 Player Goals vs. Team Goals Mismatch

In [8]:
SELECT 
    p.team_name,
    SUM(ps.goals) AS total_player_goals,
    ts.goals AS team_goals
FROM 
    Players p
JOIN 
    Player_Stats ps ON p.player_name = ps.player_name
JOIN 
    Team_Stats ts ON p.team_name = ts.team_name
GROUP BY 
    p.team_name, ts.goals
HAVING 
    SUM(ps.goals) != ts.goals;

(7 rows affected)

Total execution time: 00:00:00.139

team_name,total_player_goals,team_goals
Aston Villa,54,56
Brighton,59,60
Chelsea,59,60
Everton,37,38
Fulham,52,53
Ipswich Town,32,34
Leicester City,28,33


## 🛠 Root Cause Analysis

Possible explanations for these discrepancies:

1. Own goals (counted in team totals but not attributed to players)
    
2. Penalty shootout goals (may be recorded differently)
    
3. Data entry errors in either player or team stats
    
4. Missing players from the Players table who scored goals
    
5. Different time periods between player and team data extracts

## 🛠 5. Automated Data Quality Dashboard

In [10]:
-- Summary of Data Quality Issues
SELECT 
    'Referential Integrity' AS check_type,
    COUNT(*) AS issues_found
FROM 
    (SELECT player_name FROM Players WHERE team_name NOT IN (SELECT team_name FROM Teams)) AS ref_check
UNION ALL
SELECT 
    'Negative Stats',
    COUNT(*)
FROM 
    Player_Stats
WHERE 
    goals < 0 OR assists < 0
UNION ALL
SELECT 
    'NULL Critical Fields',
    COUNT(*)
FROM 
    Player_Stats
WHERE 
    played IS NULL OR goals IS NULL;

(3 rows affected)

Total execution time: 00:00:00.204

check_type,issues_found
Referential Integrity,0
Negative Stats,0
NULL Critical Fields,0


## 📌 Conclusion

This notebook ensures high-quality, consistent, and reliable data in the Premier League Data Warehouse.